In [1]:
import os
# Changing the working directory
os.chdir('../..')
os.getcwd()

'/home/javiermunoz/Universidad/MasterBDyDS/TFM/TFM_DSBD'

In [17]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("figure", figsize=(15, 5))

import keras
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Dropout, Flatten

import os 
from mp_loaders import load_dataset_from_pickle
from config import LM_PER_VIDEO, BASE_OUTPUT
from mp_funs import TOTAL_LANDMARKS

from sklearn.utils import shuffle

In [3]:
dataset = "top_10"
framework = "tf"
X, y, le = load_dataset_from_pickle(dataset, framework)

In [4]:
# Assign the dataset splits
X_train, X_val, X_test = X['train'], X['test'], X['val']
y_train, y_val, y_test = y['train'], y['test'], y['val']
n_classes = len(le)

# shuffling the data
X_train, y_train = shuffle(X_train, y_train)
X_val, y_val = shuffle(X_val, y_val)
X_test, y_test = shuffle(X_test, y_test)

In [5]:
# Mediapipe normalizes data for us
X_train[0].all() < 1 

True

In [6]:
X_train.shape, X_val.shape, X_test.shape

((363, 23, 258), (54, 23, 258), (84, 23, 258))

In [7]:
y_train.shape, y_val.shape, y_test.shape

((363,), (54,), (84,))

In [8]:
model = Sequential([
    LSTM(units=128, activation='relu', return_sequences=True, input_shape=(LM_PER_VIDEO, TOTAL_LANDMARKS)),
    Dropout(0.2),
    LSTM(units=256, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(units=128, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(units=64, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(units=64, activation='relu'),
    Dense(units=n_classes, activation='softmax'),
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 23, 128)           198144    
                                                                 
 dropout (Dropout)           (None, 23, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 23, 256)           394240    
                                                                 
 dropout_1 (Dropout)         (None, 23, 256)           0         
                                                                 
 lstm_2 (LSTM)               (None, 23, 128)           197120    
                                                                 
 dropout_2 (Dropout)         (None, 23, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                4

In [9]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
# CReate a customized callback so when val_acc > 94% it stops training
from keras.callbacks import EarlyStopping, ModelCheckpoint

class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True

In [11]:
epochs = 500
bst_model_path = BASE_OUTPUT + "best_weights.h5"

callbacks = [
    MyThresholdCallback(threshold=0.94),
    #ModelCheckpoint(filepath = bst_model_path, save_best_only=True, save_weights_only=True, monitor="accuracy")
]

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    callbacks=callbacks,
    batch_size=32
)

Epoch 1/500
12/12 [==============================] - 5s 221ms/step - loss: 2.2981 - accuracy: 0.1433 - val_loss: 2.2932 - val_accuracy: 0.1667
Epoch 2/500
12/12 [==============================] - 2s 132ms/step - loss: 2.2823 - accuracy: 0.1653 - val_loss: 2.2710 - val_accuracy: 0.1667
Epoch 3/500
12/12 [==============================] - 1s 117ms/step - loss: 2.2754 - accuracy: 0.1653 - val_loss: 2.2558 - val_accuracy: 0.1667
Epoch 4/500
12/12 [==============================] - 2s 190ms/step - loss: 2.2526 - accuracy: 0.1653 - val_loss: 2.2389 - val_accuracy: 0.1667
Epoch 5/500
12/12 [==============================] - 3s 251ms/step - loss: 2.2059 - accuracy: 0.1680 - val_loss: 2.2463 - val_accuracy: 0.1667
Epoch 6/500
12/12 [==============================] - 2s 179ms/step - loss: 2.1873 - accuracy: 0.1763 - val_loss: 2.1972 - val_accuracy: 0.2593
Epoch 7/500
12/12 [==============================] - 3s 282ms/step - loss: 2.1473 - accuracy: 0.2011 - val_loss: 2.1647 - val_accuracy: 0.2037

In [12]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=32)
print("test loss, test acc:", results)

Evaluate on test data
3/3 [==============================] - 0s 113ms/step - loss: 6.4526 - accuracy: 0.2738
test loss, test acc: [6.452610492706299, 0.2738095223903656]


In [2]:
SPLITS = ['train', 'val', 'test']
DATA_PATH = f"./data/videos"

for sp in SPLITS:
    for gloss in os.listdir(os.path.join(DATA_PATH, sp)):
        frames = []
        for video_name in os.listdir(os.path.join(DATA_PATH, sp, gloss)):
            if "aug" in video_name:
                os.remove(os.path.join(DATA_PATH, sp, gloss, video_name))